<a href="https://colab.research.google.com/github/xiaorui777/NLP/blob/master/Dynamic_Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dynamic Programming For Cutting Problem

### 问题描述：一段钢材，切分成不同长度能卖出不同价钱，如何切分钢材使得收益最高呢？

#### length i     1，         2，        3 ，        4，         5，         6，        7，         8 ，        9 ，        10 
#### price Pi     1，         5，        8 ，        9，        10，      17，      17，       20 ，       24，       30  

In [0]:
from collections import defaultdict

In [0]:
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30]
price = defaultdict(int)

In [0]:
for i, p in enumerate(original_price):
    price[i + 1] = p
assert price[1] == 1

In [4]:
price[132]

0

In [0]:
solution = {}
# for a given length N, we set the corrsponding split parts
# {
#    4 : (2,2)
# }

In [6]:
"""def r(n):
        return max(
        # [p[n], r(1) + r(n-1), r(2) + r(n-2), ... , r(n-1) + r(1)]   
        
        [price[n]] + [r(i) + r(n-i) for i in range(1, n)])
        
        目前的问题：如果数字很大 r(123)，需要运行很久
        时间复杂度：n！"""

'def r(n):\n        return max(\n        # [p[n], r(1) + r(n-1), r(2) + r(n-2), ... , r(n-1) + r(1)]   \n        \n        [price[n]] + [r(i) + r(n-i) for i in range(1, n)])\n        \n        目前的问题：如果数字很大 r(123)，需要运行很久\n        时间复杂度：n！'

In [0]:
from functools import lru_cache

In [0]:
@lru_cache(maxsize = 2**10)
def r(n):   
    max_price, split_point = max(
        [(price[n], 0)]+ [(r(i) + r(n-i), i) for i in range(1, n)], key=lambda x: x[0]
    )
    solution[n] = (split_point, n - split_point)
    
    return max_price

In [9]:
r(18)

52

## Dynamic Programming

（不断查表的意思）

三个要点：

+ 分析子问题的重复性
+ 子问题进行存储
+ Solution 要进行解析

In [10]:
solution

{1: (0, 1),
 2: (0, 2),
 3: (0, 3),
 4: (2, 2),
 5: (2, 3),
 6: (0, 6),
 7: (1, 6),
 8: (2, 6),
 9: (3, 6),
 10: (0, 10),
 11: (1, 10),
 12: (2, 10),
 13: (3, 10),
 14: (2, 12),
 15: (2, 13),
 16: (6, 10),
 17: (1, 16),
 18: (2, 16)}

In [0]:
def not_cut(split): return split == 0

def parse_solution(target_length, revenue_solution):
        left, right = revenue_solution[target_length]
        
        # 如果左边不切割，那么直接返回右边
        if not_cut(left): return [right]
        
        # 如果对于两边都能切分的话，就将两边的结果结合起来
        return parse_solution(left, revenue_solution) + parse_solution(right, revenue_solution)

In [12]:
parse_solution(18, solution)

[2, 6, 10]

# Dynamic Programming For Edit Distance Problem

### 问题描述：给定两个字符串 a 和 b，只允许以下三种操作：

 + 插入操作（距离定义为1）
 + 删除操作（距离定义为1）
 + 替换操作（距离定义为2）因为替换可以理解为先删除再插入

### 求：a 和 b 的最小编辑距离。

举例： "sot" => "stop"


step1 ：插入" t "。  "sot" => "stot"，编辑距离为 1。

step2 :   替换" t " 为 " p "。 "stot" => "stop"，编辑距离为 2。

即，最小编辑距离为 1 + 2 = 3

求解这个问题的思路：动态规划

In [0]:
edit_solution = {}

In [0]:
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    candidates  = [
            # string 1 delete tail
            (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)), 
            
            # string 1 add tail of string2
            (edit_distance(string1, string2[:-1]) + 1,  'ADD {}' .format(tail_s2)),
    ]
    
    # 如果 string1 和 string2 最后一位相同则加 0 ，如果不相同则加 2
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, ' ')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 2, 'SUB {} => {}'.format(tail_s1, tail_s2))
    
    candidates . append(both_forward)
    
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    edit_solution[(string1, string2)] = operation
    return min_distance

In [15]:
edit_distance('我今天确实不想吃饭', '我今天真的不太想吃饭' )

5

In [16]:
edit_distance('horse', 'ros' )

4

In [17]:
edit_distance('小明今天确实不想吃饭', '我今天真的不太想吃饭' )

8

In [18]:
edit_solution

{('h', 'r'): 'DEL h',
 ('h', 'ro'): 'DEL h',
 ('h', 'ros'): 'DEL h',
 ('ho', 'r'): 'DEL o',
 ('ho', 'ro'): ' ',
 ('ho', 'ros'): 'ADD s',
 ('hor', 'r'): ' ',
 ('hor', 'ro'): 'DEL r',
 ('hor', 'ros'): 'DEL r',
 ('hors', 'r'): 'DEL s',
 ('hors', 'ro'): 'DEL s',
 ('hors', 'ros'): ' ',
 ('horse', 'r'): 'DEL e',
 ('horse', 'ro'): 'DEL e',
 ('horse', 'ros'): 'DEL e',
 ('小', '我'): 'DEL 小',
 ('小', '我今'): 'DEL 小',
 ('小', '我今天'): 'DEL 小',
 ('小', '我今天真'): 'DEL 小',
 ('小', '我今天真的'): 'DEL 小',
 ('小', '我今天真的不'): 'DEL 小',
 ('小', '我今天真的不太'): 'DEL 小',
 ('小', '我今天真的不太想'): 'DEL 小',
 ('小', '我今天真的不太想吃'): 'DEL 小',
 ('小', '我今天真的不太想吃饭'): 'DEL 小',
 ('小明', '我'): 'DEL 明',
 ('小明', '我今'): 'DEL 明',
 ('小明', '我今天'): 'DEL 明',
 ('小明', '我今天真'): 'DEL 明',
 ('小明', '我今天真的'): 'DEL 明',
 ('小明', '我今天真的不'): 'DEL 明',
 ('小明', '我今天真的不太'): 'DEL 明',
 ('小明', '我今天真的不太想'): 'DEL 明',
 ('小明', '我今天真的不太想吃'): 'DEL 明',
 ('小明', '我今天真的不太想吃饭'): 'DEL 明',
 ('小明今', '我'): 'DEL 今',
 ('小明今', '我今'): ' ',
 ('小明今', '我今天'): 'ADD 天',
 ('小明今', '我今天真'): 'ADD 真',

In [0]:
def parse_solution(string1, string2, revenue_solution):
    
    operation = revenue_solution[string1, string2]

    if len(string1) == 1 or  len(string2) == 1: 
        if string1 != string2:
            return [string1 + ' SUB => ' + string2]
        return []
    
    if operation == ' ' and len(string1)>1 and len(string2)>1:
        return parse_solution(string1[:-1], string2[:-1],revenue_solution)
    
    if operation == ('DEL ' + string1[-1]) and len(string1)>1 and len(string2)>1:
        return [operation] + parse_solution(string1[:-1], string2,revenue_solution)
    
    if operation == ('ADD ' + string2[-1]) and len(string1)>1 and len(string2)>1:
        return [operation] + parse_solution(string1, string2[:-1],revenue_solution)
    
    return []

In [20]:
parse_solution('我今天确实不想吃饭', '我今天真的不太想吃饭' , edit_solution)

['ADD 太', 'DEL 实', 'DEL 确', 'ADD 的', 'ADD 真']

In [21]:
parse_solution('horse', 'ros' , edit_solution)

['DEL e', 'DEL r', 'h SUB => r']

In [22]:
parse_solution('小明今天确实不想吃饭', '我今天真的不太想吃饭' , edit_solution)

['ADD 太', 'DEL 实', 'DEL 确', 'ADD 的', 'ADD 真', '小明 SUB => 我']